# GDA—新型Android逆向分析与反编译工具
　　参考：[GDA官网](http://www.gda.wiki:9090)  
　　　　　[GDA文档](https://crifan.github.io/android_app_security_crack/website/)   
　　　　　[Git地址](https://github.com/charles2gan/GDA-android-reversing-Tool)  
　　　　　[GDA：中国第一款现代交互式反编译器](https://zhuanlan.zhihu.com/p/28354064)  
　　　　　[GDA：中国首款现代交互式反编译器](https://mp.weixin.qq.com/s?__biz=MzU4NzU4MDg0Mw==&mid=2247486397&idx=1&sn=80921f32136c500c020ef5ae783233b3)  
　　　　　[最简洁轻便快速的android 逆向分析工具--GDA](https://zhuanlan.zhihu.com/p/24004379)  
　　　　　[反编译工具互锤 GDA Jadx-Gui Jeb - Jadx-Gui优点](https://www.bilibili.com/read/cv13441176?spm_id_from=333.999.0.0)  
　　　　　[]()  
　　　　　[]()  

## 一、GDA简介
### 1.1、介绍
　　Gjoy Dex Analysizer(GDA)，中国第一款也是唯一一款全交互式的现代反编译器，同时也是世界上最早实现的dalvik字节码反编译器。 GDA不仅只是反编译器，同时也是一款轻便且功能强大的综合性逆向分析利器，其不依赖java且支持apk、dex、odex、oat、jar、aar、class文件的反编译， 支持python,java脚本自动化分析。GDA提供了字符串、方法、类和成员变量的交叉引用及搜索功能、代码注释功能等等。 GDA中包含多个由作者独立研究的高速分析引擎:反编译引擎、漏洞检测引擎、恶意行为检测引擎、污点传播分析引擎、反混淆引擎、apk壳检测引擎等等, 尤其是恶意行为检测引擎和污点传播引擎与反编译核心的完美融合, 大大提高了无源码逆向工程的效率, 此外反编译器还提供了很多实用功能，如路径求解、 漏洞检测、隐私泄露检测、查壳、odex转dex、oat转dex、加解密算法工具、android设备内存dump等等。在交互式分析上，提供了字符串、方法、类和域交叉引用查询、调用者查询、搜索功能、注释功能、分析结果保存等等功能。
### 1.2、GDA反编译器特性

* c++编写,独立于java和android sdk，无需安装java和android sdk即可使用  
* 有效绕过各种字节码陷阱、类型混淆以及anti-disassembling和anti-decompiling技术  
* 支持dex、odex、apk、oat、jar、class、aar文件的反编译分析  
* 支持multi-dex反编译  
* 支持对strings,class,method,field进行交叉引用和搜索(模糊匹配、精确匹配、正则)  
* 支持class、method、field及变量重命名，java代码注释  
* 分析结果保存，odex、oat转dex  
* 设备内存dump,可用于辅助脱壳  
* 反混淆支持  
* 针对字符串、方法、类、域等关键信息提供了强大的搜索、交叉引用  
* 算法工具提供主流大部分算法，可进行加密和解密  
* Python和Java自动化插件支持  
* 方法签名支持  
* 基于API链的恶意行为识别  
* 变量及寄存器追踪与溯源分析  
* 漏洞扫描，自定义漏洞规则  
* 隐私泄露检测  
* 精细化的程序路径求解  
* APK取证分析  

## 二、GDA反编译原理
　　GDA的反编译器实现采用了7阶段分析的模式，这也是一种类“ 前端 -反编译-后端”的实现方式，但与传统的反编译器实现上有所差别，同时在算法速度和反编译效果上做一些权衡，当然反编译器时刻都在进行优化和提升之中，由于反编译器的实现较为复杂，需要处理的细节比较多，所以此处仅仅是非常简单的介绍GDA的反编译核心实现，实际上也谈不上是实现原理。  

### 2.1、dex文件解析
　　dex文件解析主要用于定位类，方法，域以及字符串等等信息，其中反编译器需要用到字节码是直接从method中解析得来,此外GDA还需获取try-catch信息,调试信息等以备后用(主要在代码生成时使用)。  

### 2.2、指令解码
　　对方法的字节码进行解码，类似于反汇编，识别出240多条字节码并将其转化为低级中间表式(LIR)和高级中间表式（GDA的中间表式并非如llvm的文本型中间表示，而是内存结构型的表达式），然后对中间表式进行优化，剔除无效语句，然后生成控制流图。GDA中每个低级中间表式都一一对应着字节码指令，高级中间表式的数目小于等于低级中间表式的数目。此外，之后实现的污点传播分析引擎也是基于高级中间表达式进行的。  

### 2.3、控制流图的生成
　　扫描每个高级中间表达式，查找分支、跳转指令建立基本块儿(同时把try-catch也考虑进去)，进一步建立起控制流图。此外还需要对控制流图进行优化，简化控制流图，去除无意义的基本块，GDA还在这个阶段做一些反混淆的工作，用于对抗一些无用跳转，但是此处作用有限，更进一步的反混淆在数据流分析中实现。  

### 2.4、数据流分析
　　有了控制流图，使用DFS遍历控制流图对基本块儿内及块间进行数据流分析，本处GDA并没有采用du（定义使用链）来实现，而是采用了比较快的使用定义计数法来实现数据流分析，对低级中间代码进行优化。此外，数据流分析中，GDA同时实现了类型推断，以给每个低级中间表式的输入输出值确定其类型。同时，反混淆也在数据流分析中实现。  

### 2.5、高级中间代码生成
　　优化完成后，就可以生成高级中间代码，按照DFS遍历每个基本块，以基本块为单位进行高级中间代码生成，此处需要应用复制传播来进行低级中间表式的迭代，以生成大幅缩减的代码语句。同时应用调试信息来修改变量符号。  
### 2.6、结构化分析
　　GDA到这个阶段实现了多复合条件的分析，并对二路(if-else)、多路（switch-case、try-catch）、环(while、do-while、for)结构进行了结构化分析，为最终代码输出做准备。结构化分析时对于非结构化图，只能采用goto来实现，确保反编译后逻辑正确性。  

### 2.7、java代码生成
　　最后按照java的代码格式来生成代码，尤其对于try-catch型的结构，为了防止try块儿于代码的生成以method为单位也可以以类为单位进行代码生成。GDA主要以method为单位，并将类和方法进行了分离。  

## 三、GDA使用说明
### 3.1、主界面说明
　　打开GDA并且将你所要分析apk文件拖拽到软件界面上：  
　　![工具_GDA_主界面说明.](./images/工具_GDA_主界面说明.jpg)  

　　以上标注具体含义如下：  
　　　　1.目标文件基本信息；  
　　　　2.Dex文件所使用的API  
　　　　3.查看所有字符串；  
　　　　4.查看所有被引用过的字符串；  
　　　　5.提取出Dex中的所有URL(pro Version)；  
　　　　6.查看AndroidManifest文件；  
　　　　7.十六进制查看器，可编辑数据；  
　　　　8.恶意行为扫描分析；  
　　　　9.漏洞扫描(Pro Version)；  
　　　　10.隐私泄露扫描（Pro Version);  
　　　　11.展开权限并查看权限所属模块/方法；  
　　　　12.点击进入入口函数(method)；  
　　　　13.连接android设备进行内存dump；  
　　　　14.搜索及访问记录，双击可查看；  
　　　　15.目标文件各元素的占比分布图；  
　　　　16.Dex头，没点击DexClass可展示相应的头，每个色块代表头部不同的区域，鼠标移动在其上面可以看到提示，每个偏移可右键跳转到偏移所在的数据区；  
　　　　17.加固软件识别；  
　　　　18.当点击到代码中的相关对象时，显示该对象索引值；  
　　　　19.当点击到代码中的相关对象时，显示该对象的报名信息；  
　　　　20.当点击到代码中的相关对象时，显示该对象的名称；  

### 3.2、快捷键说明

|快捷键|功能说明|
|:---:|:---|
|F5|在反编译代码与smali代码之间切换|
|X|交叉引用,定位调用者（支持字符串，类，方法，域，实用于smali和java）|
|Esc/◄/Backspace|退回到上一次访问的方法或者类|
|►|下一个访问的方法或者类|
|G|跳转到指定偏移地址的方法|
|N|光标放在变量/方法/类上，可以修改相关名称|
|S|全局搜索|
|C|添加注释，仅支持java窗口|
|鼠标双击|光标置于方法处，双击可以进入查看方法或者其他数据|
|M|光标置于指令行时编辑指令，仅支持smali窗口|
|▲|UP	把焦点放在左边树形控件上，按下up键可以访问基于内存的上一个方法|
|▼|Down	把焦点放在左边树形控件上，访问基于内存的下一个方法|
|D|Dump方法的二进制数据，仅支持smali窗口|
|Enter|对所有编辑框的输入修改生效|
|H|16进制显示窗口|
|Ctr+H|查看搜索记录|
|Ctr+A|全选|
|Ctr+C|拷贝所选内容|
|Ctr+F|查找当前窗口的字符串|
|Ctr+S|保存当前修改生成GDA数据库文件，如方法名，类名，指令修改、注释。|

### 3.3、功能介绍
#### 3.3.1、交互式操作
　　1、字符串、类、方法和域的交叉引用；  
　　2、搜索字符串、类方法和域；  
　　3、Java代码注释；  
　　4、方法、域、类的重命名；  
　　5、将分析结果存储至GDA数据库文件中；  

#### 3.3.2、辅助分析实用程序
　　1、从ODEX中提取DEX；  
　　2、从OAT中提取DEX；  
　　3、XML解码器；  
　　4、算法工具；  
　　5、设备内存导出；  

#### 3.3.3、算法工具
　　该工具支持下列算法：　　  
　　1、散列算法：md2 md4 md5 sha sha1 sha224 sha256 sha384 sha512  
　　2、对称加密：des idea rc2 rc4 rc4-40 rc2-40 rc2-64 bf cast5 aes（128192256），具有ecb cbc ofb cfb等模式，其他模式例如（cfb1 cfb8）  
　　3、RSA编码算法：base62，base64  

#### 3.3.4、新功能
　　1、采用C++开发的全新Dalvik反编译工具；  
　　2、有好的GUI界面；  
　　3、支持Python脚本；  
　　4、封装器识别；  
　　5、多DEX支持；  
　　6、反混淆支持；  
　　7、扫描恶意API链；  
　　8、等等......  